In [1]:
# Copyright (c) 2018 Jun Hirabayashi
# Released under the MIT license
# https://opensource.org/licenses/mit-license.php  

# pose class
import numpy as np
import random
np.seterr(divide='ignore', invalid='ignore')
from numpy import sin,cos
import math 
import matplotlib.pyplot as plt

#%matplotlib inline
import glob
import yaml
from collections import OrderedDict
yaml.add_constructor(yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG,
    lambda loader, node: OrderedDict(loader.construct_pairs(node)))
import os,cv2

class Poses:
    #.......................................................
    def __init__(self):
        self.yamlPaths = []
        self.allOriginalPoses=[]    # 本来の姿勢座標
        self.normalizedPoses=[]     # ノーマライズ姿勢座標
        self.normalizedSizeRatio=[] # ノーマライズ係数（スケール）
        self.normalizedCenter=[]    # ノーマライズ係数（回転中心）
        self.normalizedAngle=[]     # ノーマライズ係数（回転角）
        self.fromWhichYaml=[]       # 姿勢定義が含まれるYamlファイル絶対パス
        self.fromWhichImage=[]      # 姿勢が含まれる画像ファイル絶対パス
        self.isFurSubstitution=[]   # 置換用画像か True,False
    
    #...姿勢Yamlから姿勢群を返す........................................
    def readSinglePoseYaml(self, yamlPath):
        f=open(yamlPath,'r')
        f.readline() # 最初の２行は読み飛ばさないとエラーになる
        f.readline() # 最初の２行は読み飛ばさないとエラーになる
        aYaml = yaml.load(f)
        f.close()
        # 人数☓１８☓３がベタで並んでいる数値を、poses[人数][18][3] に変換する
        n=aYaml['sizes'][0] # 人数
        data= np.array([float(i) for i in aYaml['data'] ])
        #data=int( np.array([float(i) for i in aYaml['data'] ]) )
        points = np.array([data[i:i+3] for i in range(0, n*18*3,3)])
        poses = [points[i:i+18] for i in range(0, n*18,18)]
        return (poses, n)
        
    #.......................................................
    # 単一姿勢ファイル（複数人数OK）を読み込み、得られた姿勢をすべて登録する
    def loadSinglePoseYamlAndRegistPoses(self, yamlPath, imgPath, isFurSubstitution, isAll):
        poses, n = self.readSinglePoseYaml(yamlPath)
        currentNumOfPoses = len(self.allOriginalPoses)
        if(isAll):
            self.allOriginalPoses.extend( poses )  #　allOriginalPoses　nに結合
            for i in range(n): # ----------ここでデータ格納
                self.fromWhichYaml.append( yamlPath )        
                self.fromWhichImage.append(imgPath)
                self.isFurSubstitution.append(isFurSubstitution)# 置換用画像か、そうでないかも登録しておく
        else:
            self.allOriginalPoses.extend( [poses[0]] )  #　allOriginalPoses　に結合
            self.fromWhichYaml.append( yamlPath )        
            self.fromWhichImage.append( imgPath )
            self.isFurSubstitution.append(isFurSubstitution)# 置換用画像か、そうでないかも登録しておく 
        return currentNumOfPoses # 登録した姿勢群の最初のIDを返す
        
    #...置換用の姿勢yaml群を、ディレクトリから読み込む....................................................
    def loadPosesFromYamlDirectoryAsSubstitution(self, directoryPath, imgPath):
        # まず指定ディレクトリから、yamlファイル群のリストを作った上で
        for filenameWithPathAsString in glob.glob(directoryPath+"*.yml"):
            self.yamlPaths.append(filenameWithPathAsString)
        # 各yamlファイルについて、「置換用画像として=True」姿勢を読み込む
        for yamlPath in self.yamlPaths:  
            yamlFileName, ext = os.path.splitext( os.path.basename(yamlPath) )
            self.loadSinglePoseYamlAndRegistPoses(yamlPath, 
                                                  imgPath+yamlFileName.replace('_pose', '') +".jpg",
                                                  True, True)

    #....置換用の画像と姿勢Yamlを読み込む..........................
    def loadAndNormalizedPosesFromYamlDirectoryAsSubstitution(self, directoryPath, imgPath):
        self.loadPosesFromYamlDirectoryAsSubstitution(directoryPath, imgPath)
        self.normalizeAllPoses()
        
    #....ｓｒｃ姿勢yamlから、姿勢を読み込む...................................................
    def loadAndNormalizePoseFromYamlFileAsSrc(self, yamlPath, imgPath):
        id = self.loadSinglePoseYamlAndRegistPoses(yamlPath, imgPath,
                                                   False, # isFurSubstitution：　src画像として登録
                                                   False  # isAll: もっとも大きな人だけ処理
                                                  )        
        normalizedPose2 = self.normalizePose(self.allOriginalPoses[id])            
        self.normalizedPoses.append(normalizedPose2)
        return id 
    
    #....姿勢の正規化...................................................
    def normalizePose(self, pose):
        # rlhipを中心とした位置移動
        center=np.array([(pose[8][0]+pose[11][0])/2.0,(pose[8][1]+pose[11][1])/2.0])
        self.normalizedCenter.append(center) # rlhip中心位置の保持
        normalizedPose0=[]              # rlhip中心位置補正した点を格納する
        for point in pose:
            normalizedPose0.append(np.array([point[0],point[1]])-center)
        size=np.linalg.norm( np.array([pose[1][0],pose[1][1]])-center ) # 胴体長を基準としたサイズ移動  
        self.normalizedSizeRatio.append(size)
        normalizedPose1=[]              # rlhip中心位置補正し、サイズ調整した点を格納する
        for point in normalizedPose0:
            normalizedPose1.append(point/size)
        angle=math.atan2(normalizedPose1[1][1],normalizedPose1[1][0]) # 胴体向きを基準とした回転
        self.normalizedAngle.append(angle)
        normalizedPose2=[]              # rlhip中心位置補正し、サイズ調整し、回転補正した点を格納する
        for point in normalizedPose1:
            rotationMatrix = np.matrix( (
                ( cos(np.pi/2+angle), sin(np.pi/2+angle)),
                (-sin(np.pi/2+angle), cos(np.pi/2+angle))
            ) )
            normalizedPose2.append( np.ravel( np.dot(rotationMatrix, point.T) ) )
        return normalizedPose2

    #....全姿勢の正規化...................................
    def normalizeAllPoses(self):
        for i, pose in enumerate( self.allOriginalPoses ): # 全姿勢を登録する
            normalizedPose2 = self.normalizePose(pose)            
            self.normalizedPoses.append(normalizedPose2)

    #....姿勢を表示（デバッグ用）.....................................
    def visualizePose(self, poseId):
        pose = self.normalizedPoses[poseId]
        x = []
        y = []
        for j in range(14):
            x.append(pose[j][0])
            y.append(pose[j][1])
        fig = plt.figure()
        plt.xlim([-2,2])
        plt.ylim([-2,2])
        ax = fig.add_subplot(1,1,1)
        ax.scatter(x,y)
        ax.set_title('first scatter plot')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.plot([x[1], x[8]], [y[1], y[8]], color='r', linestyle='-', linewidth=2)
        plt.plot([x[1], x[11]], [y[1], y[11]], color='g', linestyle='-', linewidth=2)

    #......全姿勢を表示（デバッグ用）...............................
    def visualizeAllPoses(self):
        for i, pose in enumerate( self.normalizedPoses ):
            self.visualizePose(i)
    
    #......姿勢が近い別姿勢を探す..........................
    def findClosetPair(self, sourceID):
        myNormalizedPose = self.normalizedPoses[sourceID]
        closetID = 0
        closetNormSum = 10000000000000
        for i, pose in enumerate(self.normalizedPoses):
            if(self.isFurSubstitution[i] and i!=sourceID):
                normSum = 0
                for j in [1,2,3,5,6,8,11]: #range(12): #14   
                    #normSum=normSum+np.linalg.norm( pose[j]-myNormalizedPose[j] )+random.random()*0.01
                    normSum=normSum+np.linalg.norm( pose[j]-myNormalizedPose[j] )
                if(normSum<closetNormSum):
                    closetID = i
                    closetNormSum = normSum
        return closetID
    
    #.......画像を表示（デバッグ用）..................................
    def visuaizeImage(self, imageId):
        img = cv2.imread(self.fromWhichImage[imageId], cv2.IMREAD_COLOR)
        plt.figure(figsize=(6,6))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
        
    #.......（使っていない）画像IDで指定されたものを正規化画像保存................................................
    def normalizeImageAndSave(self, imageId, imgPath):
        angle = self.normalizedAngle[imageId]*180.0/np.pi+90.0
        scale = 400.0/self.normalizedSizeRatio[imageId]
        rotationMatrix = cv2.getRotationMatrix2D(center, angle, scale)
        rotationImg = cv2.warpAffine(img, rotationMatrix, size, flags=cv2.INTER_CUBIC)
        [x,y]=self.normalizedCenter[imageId]
        croppedImg = rotationImg[y-500:y+0, x-150:x+150]        
        plt.figure(figsize=(10,10))
        plt.imshow(np.array(croppedImg)) 
        cv2.imwrite(imgPath,croppedImg)

    #....画像合成関数...................................
    def composeTwoImages(self, srcImageId, replaceImageId, isVisualize, filePath):
        # 被写体画像、入れ替え用画像の読み込み
        srcImg = cv2.imread(self.fromWhichImage[srcImageId], cv2.IMREAD_COLOR) 
        replaceImg = cv2.imread(self.fromWhichImage[replaceImageId], cv2.IMREAD_COLOR) 
        
        # 入れ替え用画像の情報
        [x, y] = self.normalizedCenter[replaceImageId]
        s =      self.normalizedSizeRatio[replaceImageId]
        
        # 入れ替え用画像を読み込み
        croppedImg = replaceImg[ int(y-s) : int(y), 
                                 int(x - s/2) : int(x + s/2)]
        size = tuple( np.array( [croppedImg.shape[1], croppedImg.shape[0]] ) )
        angle = self.normalizedAngle[ replaceImageId]*180.0/np.pi + 90.0 -(self.normalizedAngle[srcImageId]*180.0/np.pi + 90.0)
        scale = 1#self.normalizedSizeRatio[srcImageId]/self.normalizedSizeRatio[replaceImageId]
        
        #　入れ替え用画像を被写体画像に合わせて回転・スケール変換する
        replaceImgCenter = tuple( np.array( [s, s] )) # 回転中心は、股中心（x=s,y=s）
        rotationMatrix = cv2.getRotationMatrix2D( replaceImgCenter, angle, scale )
        rotationImg = cv2.warpAffine( croppedImg, rotationMatrix, size, flags=cv2.INTER_CUBIC )
        
        # 被写体画像を切り抜くマスク画像、これも被写体画像に合わせて回転・スケール変換をする
        maskImg = cv2.imread("/home/jun/Documents/source/python/mask.png", cv2.IMREAD_COLOR) 
        maskImg = cv2.resize(maskImg, (croppedImg.shape[0], croppedImg.shape[1]) )

        h, w = maskImg.shape[:2]
        replaceImgCenter = tuple( np.array( [w/2, h] )) # 回転中心は、股中心（x=s,y=s）

        angle = 0.0 - (self.normalizedAngle[srcImageId]*180.0/np.pi + 90.0)
        rotationMatrix = cv2.getRotationMatrix2D( replaceImgCenter, angle, scale )
        maskImg = cv2.warpAffine( maskImg, rotationMatrix, size, flags=cv2.INTER_CUBIC )

        maskImg2 = cv2.imread("/home/jun/Documents/source/python/mask2.png", cv2.IMREAD_COLOR) 
        maskImg2 = cv2.resize(maskImg2, (croppedImg.shape[0], croppedImg.shape[1]) )
        maskImg2 = cv2.warpAffine( maskImg2, rotationMatrix, size, flags=cv2.INTER_CUBIC )
        h, w = maskImg.shape[:2]

        # 被写体画像の大きさに沿って、マスク合成用の画像を作る
        resizeRotationImg = np.zeros( (srcImg.shape[0], srcImg.shape[1], 3), np.uint8 )
                        
        # 被写体画像への合成を行うために、マスク用画像から被写体画像上へのマッピングをする
        [ x, y ] = self.normalizedCenter[ srcImageId ]
        
        s2 =      self.normalizedSizeRatio[ srcImageId ]

        pts1 = np.float32( [[0,0],[w,0],[w,h],[0,h]] )
        pts2 = np.float32( [[ int(x-s2/2),int(y-s2)],
                            [ int(x+s2/2),   int(y-s2)],
                            [ int(x+s2/2),   int(y)],
                            [ int(x-s2/2), int(y) ]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        
        # 位置マッピングを行った後の、入れ替え用画像
        overlayImg = cv2.warpPerspective(rotationImg, M,(srcImg.shape[1], srcImg.shape[0]))
        
        h, w = maskImg.shape[:2]
        pts1 = np.float32( [[0,0],[w,0],[w,h],[0,h]] )
        pts2 = np.float32( [[int(x-s2*0.55),int(y-s2*1.0)],
                            [int(x+s2*0.55), int(y-s2*1.0)],
                            [int(x+s2*0.55),   int(y-s2*0.4)],
                            [int(x-s2*0.55),int(y-s2*0.4)]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        # 画像合成用のマスク画像
        maskImg = cv2.warpPerspective(maskImg, M,(srcImg.shape[1], srcImg.shape[0]))
        maskImg2 = cv2.warpPerspective(maskImg2, M,(srcImg.shape[1], srcImg.shape[0]))

        mask = cv2.threshold(maskImg, 128, 255, cv2.THRESH_BINARY_INV)[1]
        mask = cv2.bitwise_not(mask)
        mask2 = cv2.threshold(maskImg2, 128, 255, cv2.THRESH_BINARY_INV)[1]
        mask2 = cv2.bitwise_not(mask2)

        # 入れ替え画像をまずは合成
        overlayImgWithMask = cv2.bitwise_and(overlayImg, mask)
        mask = cv2.bitwise_not(mask)
        # 被写体画像も合成
        srcImg = cv2.bitwise_and(srcImg, mask)

        result = cv2.bitwise_or(srcImg, overlayImgWithMask)
        result = cv2.bitwise_or(result, mask2)
        if(isVisualize):
            plt.figure(figsize=(6,6))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        if(filePath is not None):
            cv2.imwrite(filePath, result)

In [2]:
#openpose呼ぶための関数
import subprocess

# do openpose to images which are in specified directory
def doOpenpose( imgDir, poseDir ): # imgDir:srcDir poseDir:dst dir of poses
    # openpose command
    # binPath = './build/examples/openpose/openpose.bin --image_dir '
    binPath = '/home/jun/Documents/openpose/build/examples/openpose/openpose.bin --image_dir '
    #cmd = binPath + imgDir + ' --write_pose ' + poseDir + ' --no_display'
    cmd = binPath + imgDir + ' --write_keypoint ' + poseDir + ' --no_display'
    print(cmd)
    proc = subprocess.call( cmd.strip().split(" "))
    print(proc)
    return proc # return code

In [3]:
import os
import shutil

from bottle import route, run, template, request, static_file, url, get, post, response, error, abort, redirect, os
import sys, codecs
#import bottle.ext.sqlalchemy
import bottle_sqlalchemy
import sqlalchemy
import sqlalchemy.ext.declarative

sys.stdout = codecs.getwriter("utf-8")(sys.stdout)

# 画像保存ディレクトリ指定
def get_save_path():
    path_dir = "/static/img/"
    return path_dir

@route("/static/<filepath:path>", name="static_file")
def static(filepath):
    return static_file(filepath, root="./static")

@route("/static/img/<img_filepath:path>", name="static_img")
def static_img(img_filepath):
    return static_img(img_filepath, root="./static/img/")

@route("/")
def html_index():
    #return template("index", url=url)
    #return template('<b>Hello {{name}}</b>!', name=name)
    return '''
        <b>Hello!</b>
    '''

@get('/upload')
def upload():
    
    #             <input type="file" name="upload" accept="image/*"></br>
    return '''
<style>
label {
  color: white;  
  background-color: blue;
  padding: 20px;
  border-radius: 20px;
}
body {
-webkit-text-size-adjust: 100%;
}
</style>
        <h1  align="center">LOOK INSIDE!(Muscle)</h1>
        <center>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <input type="hidden" value="sample" name="name" /></br>
            <label>
            Capture/Chose a Image<input type="file" name="upload" accept="image/*" style="display:none;">
            </label><br><br><br><br><br><br>
            <label>
            Look inside!<input type="submit" value="Look inside!" style="display:none;">
            </label></br>
        </center>
        </form>
    '''

@route('/upload', method='POST')
def do_upload():
    name = request.forms.name
    data = request.files.get('upload', '')
    if name and data and data.file:
        if not data.filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            return 'File extension not allowed!'
        # !!!!!!! ./tmpPose/ と ./tmpImg ディレクトリを用意していくこと   !!!!!!!!!!
        saveImgPath = "/home/jun/Documents/source/python/tmpImg/"+name+".jpg"  # tmpディレクトリを作っておくこと
        data.save(saveImgPath,overwrite=True)
        
        img = cv2.imread(saveImgPath, cv2.IMREAD_COLOR) 
        if(img.shape[1]+img.shape[0]>1600):
            w = 600
            h = int(img.shape[0]*w/img.shape[1])
            img = cv2.resize(img, (w, h) )
            cv2.imwrite(saveImgPath, img)
                          
        #openpose処理
        if( doOpenpose( "/home/jun/Documents/source/python/tmpImg/", 
                        "/home/jun/Documents/source/python/tmpPose/" ) >= 0):
                        #"/home/jun/Documents/source/python/tmpPose/" ) == 0):
            srcId = aPose.loadAndNormalizePoseFromYamlFileAsSrc(
                "/home/jun/Documents/source/python/tmpPose/"+name+"_pose.yml",  # 姿勢yamlファイル
                saveImgPath)                       # 画像ファイル
            similarID = aPose.findClosetPair(srcId)
            outImgPath = "/home/jun/Documents/source/python/static/img/"+name+".jpg"
            aPose.composeTwoImages(srcId, similarID, False, outImgPath)
            #画像削除して、処理後画像に、リダイレクト
            #os.remove(saveImgPath)
            redirect("/static/img/"+name+".jpg", 301)
        else:
            #os.remove(saveImgPath)
            # !!!!!!!./static/img/ にFAILED.jpg　を用意しておく!!!!!!!!!
            redirect("/static/img/FAILED.jpg", 301)
    return "You missed a field."

In [ ]:
# サーバ側起動スクリプト
# 実行時注意
# ①jupyter 実行Dirの下に
# tmpImg tmpPoseディレクトリを作成すること
# ②下記部分で、既存データへのパスを指定しておくこと
#aPose.loadAndNormalizedPosesFromYamlDirectoryAsSubstitution(
#      "/Users/jun/Desktop/20170521/testPoses/", # 姿勢Yaml群ディレクトリ 
#      "/Users/jun/Desktop/20170521/testImages/")# 画像ファイル群ディレクトリ
# ③実行Dieに"FAILED.jpg"を作成しておくこと

#Pose開始
aPose = Poses()
#既存データ読み込み
aPose.loadAndNormalizedPosesFromYamlDirectoryAsSubstitution(
      "/home/jun/Documents/openpose/poseImgDirs/cosplay_marvel_yaml/", # 姿勢Yaml群ディレクトリ 
      "/home/jun/Documents/openpose/poseImgDirs/cosplay_marvel_jpeg/")# 画像ファイル群ディレクトリ        
#サーバ開始（入れ替え用画像処理は、サーバ処理内で実行する）
run(host='0.0.0.0', port=8080) # jupyterではこっち
#run(host='localhost', port=8000, debug=True, reloader=True)